In [1]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords

# Chargement du dataframe créé (data_viz)
df = pd.read_csv('/mnt/c/Users/steph/OneDrive/Images/Rakuten/X_train_bis.csv') 

df.head()

,designation,description,productid,imageid,image_name,prdtypecode
0,Olivia: Personalisiertes Notizbuch / 150 Seite...,NaN,3804725264,1263597046,image_1263597046_product_3804725264.jpg,10
1,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,NaN,436067568,1008141237,image_1008141237_product_436067568.jpg,2280
2,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,PILOT STYLE Touch Pen de marque Speedlink est ...,201115110,938777978,image_938777978_product_201115110.jpg,50
3,Peluche Donald - Europe - Disneyland 2000 (Mar...,NaN,50418756,457047496,image_457047496_product_50418756.jpg,1280
4,La Guerre Des Tuques,Luc a des id&eacute;es de grandeur. Il veut or...,278535884,1077757786,image_1077757786_product_278535884.jpg,2705


Création d'un dictionnaire avec la correspondance des Labels avec la colonne 'prdtypecode'

In [2]:
dict_prdtypecode = {
    "prdtypecode": [10, 40, 50, 60, 1140, 1160, 1180, 1280, 1281, 1300, 1301, 1302, 1320,
                    1560, 1920, 1940, 2060, 2220, 2280, 2403, 2462, 2522, 2582, 2583,
                    2585, 2705, 2905],

    "Label": ["Livres adultes", "Jeux Vidéos", "Accessoires de Jeux Vidéos", "Consoles de jeux", "Figurine", "Carte à Collectionner",
              "Masques", "Jouets pour Enfants", "Jeux de Cartes et de société", "Produits télécommandés",
              "Vêtements pour enfants", "Jouets pour Enfants", "Produits Bébés et Enfants",
              "Literies et Meubles", "Accessoires Maison", "Alimentation", "Décoration d'intérieur", "Accessoires Animaux",
              "Journaux et Magazines", "Livres et Revues", "Jeux", "Papeterie",
              "Mobilier", "Piscine", "Jardinage", "Livres", "Jeux en ligne et Logiciels"]
}


Ajout de la colonne 'Label' au dataframe df pour identifier le nom de chaque classes

In [3]:
# création d'un dataframe avec le dictionnaire dict_prdtypecode
df_class = pd.DataFrame(data=dict_prdtypecode)

# création de la correspondance entre "prdtypecode" et "Label"
dict_code_label = dict(zip(df_class['prdtypecode'], df_class['Label']))

# ajout de la colonne 'Label' au dataframe
df['Label'] = df['prdtypecode'].replace(dict_code_label)


Ajout d'une nouvelle colonne "prdtypecode_base" (codes produits initiaux) 

In [4]:
df["prdtypecode_base"] = df["prdtypecode"]

Remplacement des codes produits dans la colonne 'prdtypecode' par les 27 classes correspondantes

In [5]:
df['prdtypecode'].replace({10: 0, 40: 1, 50: 2, 60: 3, 1140: 4, 1160: 5, 1180: 6, 1280: 7, 1281: 8,
                           1300: 9, 1301: 10, 1302: 11, 1320: 12, 1560: 13, 1920: 14, 1940: 15, 2060: 16,
                           2220: 17, 2280: 18, 2403: 19, 2462: 20, 2522: 21, 2582: 22, 2583: 23,
                           2585: 24, 2705: 25, 2905: 26}, inplace=True)

In [6]:
df.head()

,designation,description,productid,imageid,image_name,prdtypecode,Label,prdtypecode_base
0,Olivia: Personalisiertes Notizbuch / 150 Seite...,NaN,3804725264,1263597046,image_1263597046_product_3804725264.jpg,0,Livres adultes,10
1,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,NaN,436067568,1008141237,image_1008141237_product_436067568.jpg,18,Journaux et Magazines,2280
2,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,PILOT STYLE Touch Pen de marque Speedlink est ...,201115110,938777978,image_938777978_product_201115110.jpg,2,Accessoires de Jeux Vidéos,50
3,Peluche Donald - Europe - Disneyland 2000 (Mar...,NaN,50418756,457047496,image_457047496_product_50418756.jpg,7,Jouets pour Enfants,1280
4,La Guerre Des Tuques,Luc a des id&eacute;es de grandeur. Il veut or...,278535884,1077757786,image_1077757786_product_278535884.jpg,25,Livres,2705


# Preprocessing

In [7]:
nltk.download('punkt')
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
stop_words = stopwords.words('french') + stopwords.words('english') + stopwords.words('german') +[ \
                'plus', 'peut', 'tout', 'etre', 'sans', 'dont', 'aussi',  \
                  'comme', 'meme', 'bien','leurs', 'elles', 'cette','celui',   \
                  'ainsi', 'encore', 'alors', 'toujours', 'toute','deux', 'nouveau',   \
                  'peu', 'car', 'autre', 'jusqu', 'quand', 'ici', 'ceux', 'enfin',  \
                  'jamais', 'autant', 'tant', 'avoir', 'moin', 'celle', 'tous',   \
                  'contre', 'pourtant', 'quelque', 'toutes', 'surtout', 'cet',  \
                  'comment', 'rien', 'avant', 'doit', 'autre', 'depuis', 'moins',  \
                  'tre', 'souvent', 'etait', 'pouvoir', 'apre', 'non', 'ver', 'quel',   \
                  'pourquoi', 'certain', 'fait', 'faire', 'sou', 'donc', 'trop',  \
                  'quelques', 'parfois', 'tres', 'donc', 'dire', 'eacute', 'egrave',  \
                  'rsquo', 'agrave', 'ecirc', 'nbsp', 'acirc', 'apres', 'autres',  \
                  'ocirc', 'entre', 'sous', 'quelle']

def CreateTextANDcleaning(data):
    "fonction pour créer une colonne 'text' nettoyée comprenant les colonnes 'designation' et 'description'"
    
    df = data
    # conversion en chaîne de caractères 
    df['designation'] = df['designation'].astype('string')
    df['description'] = df['description'].astype('string')

    # remplacement des Nan par une chaîne de caractère
    df['description'].fillna('', inplace=True)
 
    # création d'une colonne 'text' contenant description et désignation
    df['text']=""
    for i in range(df.shape[0]):
        df['text'][i] = create_text(df['designation'][i], df['description'][i])
  
    df['text'] = df['text'].str.split()
  
    df['text'] = df['text'].apply(lambda x: unique_description(x))
    df['text'] = df['text'].apply(lambda x: " ".join(x))
  
    df['text'] = df['text'].apply(lambda text : lower_case(text))
    df['text'] = df['text'].apply(lambda text : remove_accent(text))  
  
    df['text'] = df['text'].apply(lambda text : remove_htmltags(text))
    df['text'] = df['text'].apply(lambda text : keeping_essentiel(text))
  
    # Divise chaque chaîne de texte en une liste de mots en une seule chaîne de caractères en appliquant les fonctions précédentes  
    df['text'] = df['text'].str.split()  
    df['text']= df['text'].apply(lambda x: operation(x))
      
    df['text'] = df['text'].apply(lambda x: " ".join(x))   
      
    return df['text']


def create_text(text1, text2):
    """
    fonction prend deux chaînes de caractères en entrée (text1 et text2) et les concatène.
    Si text2 est manquant, alors seulement text1 est retourné.
    """
    if pd.isna(text2):
        text = text1
    else:
        text = text1 +" "+ text2
    return text

def unique_description(text):
    unique=[text[0]]
    for mot in text:
        if mot not in unique:
            unique.append(mot)
    return unique

def lower_case(text):
    text = text.lower().strip()
    return text

def remove_accent(string):
    string = string.replace('á', 'a')
    string = string.replace('â', 'a')

    string = string.replace('é', 'e')
    string = string.replace('è', 'e')
    string = string.replace('ê', 'e')
    string = string.replace('ë', 'e')

    string = string.replace('î', 'i')
    string = string.replace('ï', 'i')

    string = string.replace('ö', 'o')
    string = string.replace('ô', 'o')
    string = string.replace('ò', 'o')
    string = string.replace('ó', 'o')

    string = string.replace('ù', 'u')
    string = string.replace('û', 'u')
    string = string.replace('ü', 'u')

    string = string.replace('ç', 'c')
    
    return string

def remove_htmltags(text):
    text = re.sub('<[^<]+?>', '',text)
    return text

def keeping_essentiel(text):
    text = re.sub(r"[^a-zA-Z]+", " ", text)
    return text

def operation(x):
    """
    fonction prend une liste de mots en entrée (x) et renvoie une nouvelle liste 
    qui ne contient que les mots qui ne sont pas dans la liste des mots d'arrêt 
    et qui ont plus de 2 caractères.
    """
    list=[]
    for word in x:
        if (word not in stop_words and len(word)>2):
            list.append(word)
    return list

In [9]:
# Appel de la fonction pour vérifier le nettoyage
CreateTextANDcleaning(df)

/tmp/ipykernel_194452/854338298.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i] = create_text(df['designation'][i], df['description'][i])


0        olivia personalisiertes notizbuch seiten punkt...
1        journal arts art marche salon art asiatique pa...
2        grand stylet ergonomique bleu gamepad nintendo...
3        peluche donald europe disneyland marionnette d...
4        guerre tuques luc grandeur veut organiser jeu ...
                               ...                        
84911                                  sims import anglais
84912    kit piscine acier nevada deco pierre descripti...
84913    journal officiel republique francaise changeme...
84914    table basse bois recuperation massif base blan...
84915    gomme collection gommes pinguin glace vert orange
Name: text, Length: 84916, dtype: object

In [10]:
df.head()

,designation,description,productid,imageid,image_name,prdtypecode,Label,prdtypecode_base,text
0,Olivia: Personalisiertes Notizbuch / 150 Seite...,,3804725264,1263597046,image_1263597046_product_3804725264.jpg,0,Livres adultes,10,olivia personalisiertes notizbuch seiten punkt...
1,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,,436067568,1008141237,image_1008141237_product_436067568.jpg,18,Journaux et Magazines,2280,journal arts art marche salon art asiatique pa...
2,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,PILOT STYLE Touch Pen de marque Speedlink est ...,201115110,938777978,image_938777978_product_201115110.jpg,2,Accessoires de Jeux Vidéos,50,grand stylet ergonomique bleu gamepad nintendo...
3,Peluche Donald - Europe - Disneyland 2000 (Mar...,,50418756,457047496,image_457047496_product_50418756.jpg,7,Jouets pour Enfants,1280,peluche donald europe disneyland marionnette d...
4,La Guerre Des Tuques,Luc a des id&eacute;es de grandeur. Il veut or...,278535884,1077757786,image_1077757786_product_278535884.jpg,25,Livres,2705,guerre tuques luc grandeur veut organiser jeu ...


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84916 entries, 0 to 84915
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   designation       84916 non-null  string
 1   description       84916 non-null  string
 2   productid         84916 non-null  int64 
 3   imageid           84916 non-null  int64 
 4   image_name        84916 non-null  object
 5   prdtypecode       84916 non-null  int64 
 6   Label             84916 non-null  object
 7   prdtypecode_base  84916 non-null  int64 
 8   text              84916 non-null  object
dtypes: int64(4), object(3), string(2)
memory usage: 5.8+ MB


Sauvegarde du dataframe nettoyé

In [13]:
# Enregistrement du nouveau dataframe nettoyé
path = '/mnt/c/Users/steph/OneDrive/Images/Rakuten/df_cleaned.csv'
df.to_csv(path, index=False)